In [115]:
import glob
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
from src.func_data_engineering import *
from src.func_data_diagnosis import *

pd.set_option('display.max_columns', None)

# auto load modules
%load_ext autoreload
%autoreload

%cd C:\Users\tyler\repos\airbnb-forecast

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\tyler\repos\airbnb-forecast


In [109]:
# read listing data
path = r'data\test'
listing_name = "listings*.csv"
cols = [
    # listing
    'id', 'last_scraped', 'property_type', 'room_type', 'accommodates'
    , 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet'
    , 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy'
    , 'require_guest_profile_picture', 'require_guest_phone_verification'
    , 'guests_included'
    # calendar
    , 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights'
    , 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm'
    , 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30'
    , 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped'
    # reviews
    , 'number_of_reviews', 'number_of_reviews_ltm', 'first_review','last_review'
    , 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness'
    , 'review_scores_checkin', 'review_scores_communication', 'review_scores_location'
    , 'review_scores_value', 'reviews_per_month'
    # prices
    , 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee'
    , 'guests_included', 'extra_people'
    # location
    , 'street','neighbourhood_cleansed', 'city', 'state'
    , 'zipcode', 'market', 'smart_location', 'country', 'latitude', 'longitude'
    , 'is_location_exact'
    # host
    , 'host_id', 'host_since', 'host_neighbourhood', 'host_response_time'
    , 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost'
    , 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes'
    , 'calculated_host_listings_count_private_rooms'
    , 'calculated_host_listings_count_shared_rooms'
    , 'host_verifications', 'host_has_profile_pic', 'host_identity_verified'
    ]

listings = read_data(path,listing_name, 'last_scraped', cols )
df_listing = pd.concat(listings,axis=0,ignore_index=True)

In [48]:
# read calendar data
cal_name = "calendar*.csv"
calendars = read_data(path, cal_name, 'date')

# for each monthly scrape, only keep the calendar data b/w the scraped date and the date of next scrape run
cal_months = []
num_cal = len(calendars)
for i in range(num_cal):
    if i < num_cal - 1:
        date_end = calendars[i + 1].SCRAPED_DATE[0]
        df = calendars[i]
        df = df[df.date < date_end]
    else:
        df = calendars[i]
        date_start = pd.to_datetime(df.SCRAPED_DATE[0]).date()
        date_end = date_start + relativedelta.relativedelta(months=1)
        df = df[df.date < str(date_end)]
    cal_months.append(df)

df_cal = pd.concat(cal_months, axis=0, ignore_index=True)

In [110]:
# cleanse
df_listing = cleanse_data(df_listing)
df_listing.info()

C:\Users\tyler\Anaconda3\lib\site-packages\pandas\core\series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77121 entries, 0 to 77120
Data columns (total 77 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                77121 non-null  int64  
 1   LAST_SCRAPED                      77121 non-null  object 
 2   PROPERTY_TYPE                     77121 non-null  object 
 3   ROOM_TYPE                         77121 non-null  object 
 4   ACCOMMODATES                      77121 non-null  int64  
 5   BATHROOMS                         77062 non-null  float64
 6   BEDROOMS                          77018 non-null  float64
 7   BEDS                              76582 non-null  float64
 8   BED_TYPE                          77121 non-null  object 
 9   AMENITIES                         77121 non-null  object 
 10  SQUARE_FEET                       534 non-null    float64
 11  INSTANT_BOOKABLE                  77121 non-null  object 
 12  IS_B

In [61]:
df_cal = cleanse_data(df_cal, list_data=False)
df_cal.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574048 entries, 0 to 2574047
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              int64  
 1   DATE            object 
 2   AVAILABLE       object 
 3   BASE_PRICE      float64
 4   TXN_PRICE       float64
 5   MINIMUM_NIGHTS  float64
 6   MAXIMUM_NIGHTS  float64
 7   SCRAPED_DATE    object 
 8   BOOKED          int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 176.7+ MB


In [98]:
df_cal = agg_to_monthly(df_cal)
df_cal.shape

(106314, 5)

In [111]:
df_listing['YEAR_MONTH'] = df_listing.SCRAPED_DATE.str[:7]
df_data = df_cal.merge(df_listing, on = ['ID', 'YEAR_MONTH'])
df_data.shape

(77121, 81)

In [90]:
df_list = fs_listing(df_data,output_all=True)
df_list.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77121 entries, 0 to 77120
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                77121 non-null  int64  
 1   YEAR_MONTH                        77121 non-null  object 
 2   PROPERTY_TYPE                     77121 non-null  object 
 3   ROOM_TYPE                         77121 non-null  object 
 4   ACCOMMODATES                      77121 non-null  int64  
 5   BATHROOMS                         77062 non-null  float64
 6   BEDROOMS                          77018 non-null  float64
 7   BEDS                              76582 non-null  float64
 8   BED_TYPE                          77121 non-null  object 
 9   AMENITIES                         77121 non-null  object 
 10  SQUARE_FEET                       534 non-null    float64
 11  INSTANT_BOOKABLE                  77121 non-null  int64  
 12  IS_B

In [93]:
df_price = fs_price(df_data, monthly=True)
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77121 entries, 0 to 77120
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 77121 non-null  int64  
 1   YEAR_MONTH         77121 non-null  object 
 2   TXN_PRICE          77115 non-null  float64
 3   PRICE_PER_GUEST    77115 non-null  float64
 4   SECURITY_DEPOSIT   54505 non-null  object 
 5   CLEANING_FEE       61048 non-null  object 
 6   EXTRA_PEOPLE       77121 non-null  object 
 7   PRICE_LAG_1        48005 non-null  float64
 8   PRICE_LAG_2        22502 non-null  float64
 9   PRICE_LAG_3        0 non-null      float64
 10  PRICE_MINUS_LAG_1  48005 non-null  float64
 11  PRICE_MINUS_LAG_2  22502 non-null  float64
 12  PRICE_MINUS_LAG_3  0 non-null      float64
 13  PRICE_MA_3         22502 non-null  float64
 14  PRICE_MINUS_MA_3   22502 non-null  float64
dtypes: float64(10), int64(1), object(4)
memory usage: 9.4+ MB


In [102]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77121 entries, 0 to 77120
Data columns (total 80 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                77121 non-null  int64  
 1   YEAR_MONTH                        77121 non-null  object 
 2   BASE_PRICE                        77115 non-null  float64
 3   TXN_PRICE                         77115 non-null  float64
 4   BOOKED                            77121 non-null  int64  
 5   LAST_SCRAPED                      77121 non-null  object 
 6   PROPERTY_TYPE                     77121 non-null  object 
 7   ROOM_TYPE                         77121 non-null  object 
 8   ACCOMMODATES                      77121 non-null  int64  
 9   BATHROOMS                         77062 non-null  float64
 10  BEDROOMS                          77018 non-null  float64
 11  BEDS                              76582 non-null  float64
 12  BED_

In [95]:
df_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574048 entries, 0 to 2574047
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID              int64  
 1   DATE            object 
 2   AVAILABLE       object 
 3   BASE_PRICE      float64
 4   TXN_PRICE       float64
 5   MINIMUM_NIGHTS  float64
 6   MAXIMUM_NIGHTS  float64
 7   SCRAPED_DATE    object 
 8   BOOKED          int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 176.7+ MB


In [96]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77121 entries, 0 to 77120
Data columns (total 77 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                77121 non-null  int64  
 1   LAST_SCRAPED                      77121 non-null  object 
 2   PROPERTY_TYPE                     77121 non-null  object 
 3   ROOM_TYPE                         77121 non-null  object 
 4   ACCOMMODATES                      77121 non-null  int64  
 5   BATHROOMS                         77062 non-null  float64
 6   BEDROOMS                          77018 non-null  float64
 7   BEDS                              76582 non-null  float64
 8   BED_TYPE                          77121 non-null  object 
 9   AMENITIES                         77121 non-null  object 
 10  SQUARE_FEET                       534 non-null    float64
 11  MINIMUM_NIGHTS                    77121 non-null  int64  
 12  MAXI

In [112]:
df_calendar = fs_calendar(df_data, output_all=True)
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77121 entries, 0 to 77120
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  77121 non-null  int64  
 1   YEAR_MONTH          77121 non-null  object 
 2   MAX_MINIMUM_NIGHTS  77121 non-null  int64  
 3   MIN_MINIMUM_NIGHTS  77121 non-null  int64  
 4   MIN_MAXIMUM_NIGHTS  77121 non-null  int64  
 5   MAX_MAXIMUM_NIGHTS  77121 non-null  int64  
 6   AVG_MINIMUM_NIGHTS  77121 non-null  float64
 7   AVG_MAXIMUM_NIGHTS  77121 non-null  float64
 8   CALENDAR_UPDATED    77121 non-null  object 
 9   AVAILABILITY_60     77121 non-null  int64  
 10  AVAILABILITY_90     77121 non-null  int64  
 11  AVAILABILITY_365    77121 non-null  int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 7.6+ MB


In [116]:
df_booked = fs_booked(df_data, output_all=True)
df_booked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77121 entries, 0 to 77120
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         77121 non-null  int64  
 1   YEAR_MONTH                 77121 non-null  object 
 2   BOOKED                     77121 non-null  int64  
 3   BOOKED_LAG_1               48011 non-null  float64
 4   BOOKED_LAG_3               0 non-null      float64
 5   BOOKED_LAG1_MINUS_LAG3     0 non-null      float64
 6   BOOKED_LAG1_MA3            0 non-null      float64
 7   BOOKED_LAG1_MINUS_LAG1MA3  0 non-null      float64
dtypes: float64(5), int64(2), object(1)
memory usage: 5.3+ MB


In [ ]:
engine = connect_my_db('secrets/db_string')

In [166]:
fs_upload(engine,df_ls,'FS_LIST')

85.58639979362488

In [167]:
fs_upload(engine,df_host,'FS_HOST')

73.12233901023865

In [168]:
fs_upload(engine,df_review,'FS_REVIEW')

106.92497611045837

In [169]:
fs_upload(engine,df_location,'FS_LOCATION_RAW')

81.0525414943695

In [159]:
# feature store

# FS_LISTING
df_ls = df_list[['ID','HOST_ID','LAST_SCRAPED',
                 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'INSTANT_BOOKABLE', 'IS_BUSINESS_TRAVEL_READY','GUESTS_INCLUDED',
       'CANCELLATION_POLICY', 'REQUIRE_GUEST_PROFILE_PICTURE',
       'REQUIRE_GUEST_PHONE_VERIFICATION',
       'LUXURY_FLAG','NEIGHBOURHOOD_CLEANSED','LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT',]]

df_ls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID                                148835 non-null  int64  
 1   HOST_ID                           148835 non-null  int64  
 2   LAST_SCRAPED                      148835 non-null  object 
 3   PROPERTY_TYPE                     148835 non-null  object 
 4   ROOM_TYPE                         148835 non-null  object 
 5   ACCOMMODATES                      148835 non-null  int64  
 6   BATHROOMS                         148718 non-null  float64
 7   BEDROOMS                          148606 non-null  float64
 8   BEDS                              147612 non-null  float64
 9   BED_TYPE                          148835 non-null  object 
 10  AMENITIES                         148835 non-null  object 
 11  SQUARE_FEET                       1044 non-null    f

In [128]:
# FS_HOST


df_host = df_list[['ID', 'LAST_SCRAPED', 'HOST_ID', 'HOST_SINCE', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT',
       'HOST_ENTIRE_HOMES',
       'HOST_PRIVATE_ROOMS',
       'HOST_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED']]

df_host.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      148835 non-null  int64  
 1   LAST_SCRAPED            148835 non-null  object 
 2   HOST_ID                 148835 non-null  int64  
 3   HOST_SINCE              148751 non-null  object 
 4   HOST_NEIGHBOURHOOD      94904 non-null   object 
 5   HOST_RESPONSE_TIME      99604 non-null   object 
 6   HOST_RESPONSE_RATE      99604 non-null   float64
 7   HOST_ACCEPTANCE_RATE    93013 non-null   float64
 8   HOST_IS_SUPERHOST       148751 non-null  float64
 9   HOST_LISTINGS_COUNT     148835 non-null  int64  
 10  HOST_ENTIRE_HOMES       148835 non-null  int64  
 11  HOST_PRIVATE_ROOMS      148835 non-null  int64  
 12  HOST_SHARED_ROOMS       148835 non-null  int64  
 13  HOST_VERIFICATIONS      148835 non-null  object 
 14  HOST_HAS_PROFILE_PIC

In [144]:
# fs reviews

df_review = df_list[['ID', 'LAST_SCRAPED','NUMBER_OF_REVIEWS',
                     'FIRST_REVIEW',
       'LAST_REVIEW', 'REVIEW_SCORES_RATING', 'REVIEW_SCORES_ACCURACY',
       'REVIEW_SCORES_CLEANLINESS', 'REVIEW_SCORES_CHECKIN',
       'REVIEW_SCORES_COMMUNICATION', 'REVIEW_SCORES_LOCATION',
       'REVIEW_SCORES_VALUE','REVIEWS_PER_MONTH']]

df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           148835 non-null  int64  
 1   LAST_SCRAPED                 148835 non-null  object 
 2   NUMBER_OF_REVIEWS            148835 non-null  int64  
 3   FIRST_REVIEW                 118283 non-null  object 
 4   LAST_REVIEW                  118283 non-null  object 
 5   REVIEW_SCORES_RATING         115283 non-null  float64
 6   REVIEW_SCORES_ACCURACY       115171 non-null  float64
 7   REVIEW_SCORES_CLEANLINESS    115204 non-null  float64
 8   REVIEW_SCORES_CHECKIN        115123 non-null  float64
 9   REVIEW_SCORES_COMMUNICATION  115189 non-null  float64
 10  REVIEW_SCORES_LOCATION       115129 non-null  float64
 11  REVIEW_SCORES_VALUE          115119 non-null  float64
 12  REVIEWS_PER_MONTH            118283 non-null  float64
dtyp

In [170]:
# fs location

df_location = df_list[['ID', 'SCRAPED_DATE','STREET','NEIGHBOURHOOD_CLEANSED','CITY', 'STATE', 'ZIPCODE', 'MARKET',
       'SMART_LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT']]

KeyError: "['SCRAPED_DATE'] not in index"

In [165]:
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148835 entries, 0 to 148834
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   STREET                  148835 non-null  object 
 1   NEIGHBOURHOOD_CLEANSED  148835 non-null  object 
 2   CITY                    148739 non-null  object 
 3   STATE                   148261 non-null  object 
 4   ZIPCODE                 148238 non-null  object 
 5   MARKET                  148669 non-null  object 
 6   SMART_LOCATION          148835 non-null  object 
 7   COUNTRY                 148835 non-null  object 
 8   LATITUDE                148835 non-null  float64
 9   LONGITUDE               148835 non-null  float64
 10  IS_LOCATION_EXACT       148835 non-null  int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 12.5+ MB


In [129]:
df_list.columns

Index(['ID', 'LAST_SCRAPED', 'HOST_ID', 'HOST_SINCE', 'HOST_NEIGHBOURHOOD',
       'HOST_RESPONSE_TIME', 'HOST_RESPONSE_RATE', 'HOST_ACCEPTANCE_RATE',
       'HOST_IS_SUPERHOST', 'HOST_LISTINGS_COUNT', 'HOST_ENTIRE_HOMES',
       'HOST_PRIVATE_ROOMS', 'HOST_SHARED_ROOMS', 'HOST_VERIFICATIONS',
       'HOST_HAS_PROFILE_PIC', 'HOST_IDENTITY_VERIFIED', 'STREET',
       'NEIGHBOURHOOD_CLEANSED', 'CITY', 'STATE', 'ZIPCODE', 'MARKET',
       'SMART_LOCATION', 'COUNTRY', 'LATITUDE', 'LONGITUDE',
       'IS_LOCATION_EXACT', 'PROPERTY_TYPE', 'ROOM_TYPE', 'ACCOMMODATES',
       'BATHROOMS', 'BEDROOMS', 'BEDS', 'BED_TYPE', 'AMENITIES', 'SQUARE_FEET',
       'PRICE', 'WEEKLY_PRICE', 'MONTHLY_PRICE', 'SECURITY_DEPOSIT',
       'CLEANING_FEE', 'GUESTS_INCLUDED', 'EXTRA_PEOPLE', 'MINIMUM_NIGHTS',
       'MAXIMUM_NIGHTS', 'MINIMUM_MINIMUM_NIGHTS', 'MAXIMUM_MINIMUM_NIGHTS',
       'MINIMUM_MAXIMUM_NIGHTS', 'MAXIMUM_MAXIMUM_NIGHTS',
       'MINIMUM_NIGHTS_AVG_NTM', 'MAXIMUM_NIGHTS_AVG_NTM', 'CALENDAR_UP

In [ ]:
# upload FS_LIST_RAW

fs_upload(engine,df_list,'FS_LIST_RAW')

In [ ]:
# features for model

col_list = ['id', 'last_scraped', 'host_since','host_neighbourhood',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'calculated_host_listings_count',
         'host_verifications','host_has_profile_pic', 'host_identity_verified', 
         'street','neighbourhood_cleansed',
        'city', 'state', 'zipcode', 'market', 'country', 'latitude', 'longitude',
       'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 
       'price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'calendar_updated',
       'number_of_reviews','number_of_reviews_ltm', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value','instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 
        'reviews_per_month']

In [29]:
df_cal.head()

,LISTING_ID,DATE,AVAILABLE,PRICE,ADJUSTED_PRICE,MINIMUM_NIGHTS,MAXIMUM_NIGHTS,BOOKED
0,251965,2020-01-07,f,259.0,259.0,1.0,365.0,1
1,653384,2020-01-07,f,100.0,100.0,1.0,1125.0,1
2,653384,2020-01-08,f,100.0,100.0,1.0,1125.0,1
3,653384,2020-01-09,f,100.0,100.0,1.0,1125.0,1
4,653384,2020-01-10,f,100.0,100.0,1.0,1125.0,1


In [43]:
df_cal.shape

(4555609, 9)

In [28]:
# upload FS_CAL_RAW

engine = connect_my_db('secrets/db_string')
fs_upload(engine,df_cal,'FS_CAL_RAW')

1143.5180366039276

In [56]:
df_month

,LISTING_ID,YEAR_MONTH,ADJUSTED_PRICE,BOOKED
0,9835,2020-01,60.0,0
1,9835,2020-02,60.0,0
2,9835,2020-03,60.0,0
3,9835,2020-04,60.0,0
4,9835,2020-05,60.0,0
...,...,...,...,...
180369,43733191,2020-06,320.0,11
180371,43744200,2020-06,55.0,15
180373,43746933,2020-06,140.0,7
180375,43748923,2020-06,79.0,19


In [ ]:
# upload FS_CAL_MONTHLY

upload_df(engine,df_month,'FS_CAL_MONTHLY')

In [ ]:
fre=cal.listing_id.value_counts()

In [ ]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

In [ ]:
cal.date.max()

In [ ]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

In [ ]:
fre.unique()

In [6]:
fre.unique()

In [9]:
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,t,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,f,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,f,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,f,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,f,$45.00,$45.00,1.0,1125.0


In [10]:
fre=cal.listing_id.value_counts()

In [19]:
cal.available=cal.available.replace({'t':1,'f':0})
cal.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,323280,2020-06-12,1,$67.00,$67.00,1.0,730365.0
1,315513,2020-06-12,0,$45.00,$45.00,1.0,1125.0
2,315513,2020-06-13,0,$45.00,$45.00,1.0,1125.0
3,315513,2020-06-14,0,$45.00,$45.00,1.0,1125.0
4,315513,2020-06-15,0,$45.00,$45.00,1.0,1125.0


In [21]:
cal.date.max()

'2021-06-13'

In [20]:
cal[['listing_id','available']].groupby(['listing_id']).sum()

,available
listing_id,
9835,365
10803,41
12936,0
38271,351
41836,0
...,...
43733191,79
43744200,4
43746933,173


In [15]:
fre.unique()

array([367, 366, 365], dtype=int64)